## Documentation

To read more about field data types, visit the [docs](https://www.elastic.co/guide/en/elasticsearch/reference/current/mapping-types.html).

![field_data_types_docs](../images/field_data_types_docs.png)

## Connect to ElasticSearch

In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

HOST = "http://localhost:9200"

es = Elasticsearch(HOST)
client_info = es.info()
print("Connected tp Elasticsearch!")
pprint(client_info.body)

Connected tp Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'a-_W5fVITCO39W2cUo2r2Q',
 'name': 'e3e67a410071',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-01-09T14:09:01.578835424Z',
             'build_flavor': 'default',
             'build_hash': '0f88dde84795b30ca0d2c0c4796643ec5938aeb5',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '8.11.3',
             'minimum_index_compatibility_version': '6.0.0-beta1',
             'minimum_wire_compatibility_version': '6.8.0',
             'number': '7.17.27'}}


/tmp/ipykernel_13388/13932774.py:7: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client_info = es.info()


## 1. Common types

### 1.1 Binary

In [2]:
BINARY_INDEX = "binary_index"

mapping = {
    "properties": {
        "image_data": {
            "type": "binary"
        }
    }
}

settings = {
    "index": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    }
}

es.indices.delete(index=BINARY_INDEX, ignore_unavailable=True)
es.indices.create(index=BINARY_INDEX, mappings=mapping, settings=settings)

/tmp/ipykernel_13388/708971896.py:18: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index=BINARY_INDEX, ignore_unavailable=True)
/tmp/ipykernel_13388/708971896.py:19: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index=BINARY_INDEX, mappings=mapping, settings=settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'binary_index'})

In [3]:
import base64

image_path = "../images/field_data_types_docs.png"
with open(image_path, "rb") as image_file:
    image_bytes = image_file.read()
    image_base64 = base64.b64encode(image_bytes).decode("utf-8")

image_base64[:100]

'iVBORw0KGgoAAAANSUhEUgAAB4AAAAJTCAYAAADpMAvgAAAABHNCSVQICAgIfAhkiAAAABl0RVh0U29mdHdhcmUAZ25vbWUtc2Ny'

In [4]:
document = {
    "image_data": image_base64
}
response = es.index(index=BINARY_INDEX, body=document)
response

/tmp/ipykernel_13388/2080750289.py:4: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.index(index=BINARY_INDEX, body=document)


ObjectApiResponse({'_index': 'binary_index', '_type': '_doc', '_id': 'qacBI5UB6zEJPjitnKTc', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

### 1.2 Others

In [5]:
OTHER_COMMON_DTYPE_INDEX = "other_common_data_types_index"

mappings = {
    "properties": {
        "book_reference": {
            "type": "keyword"
        },
        "price": {
            "type": "float"
        },
        "publish_date": {
            "type": "date"
        },
        "is_available": {
            "type": "boolean"
        },
    }
}

es.indices.delete(index=OTHER_COMMON_DTYPE_INDEX, ignore_unavailable=True)
es.indices.create(index=OTHER_COMMON_DTYPE_INDEX, mappings=mappings, settings=settings)

/tmp/ipykernel_13388/769605474.py:20: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index=OTHER_COMMON_DTYPE_INDEX, ignore_unavailable=True)
/tmp/ipykernel_13388/769605474.py:21: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index=OTHER_COMMON_DTYPE_INDEX, mappings=mappings, settings=settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'other_common_data_types_index'})

In [6]:
document = {
    "book_reference": "978-1617294433",
    "price": 44.99,
    "publish_date": "2021-06-30",
    "is_available": True
}

response = es.index(index=OTHER_COMMON_DTYPE_INDEX, body=document)
response

/tmp/ipykernel_13388/2445897368.py:8: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.index(index=OTHER_COMMON_DTYPE_INDEX, body=document)


ObjectApiResponse({'_index': 'other_common_data_types_index', '_type': '_doc', '_id': 'qqcFI5UB6zEJPjitpqTF', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

## 2. Object types

### 2.1 Object

In [7]:
OBJECT_INDEX = "object_index"

mappings = {
    "properties": {
        "author": {
            "properties": {
                "first_name": {
                    "type": "text"
                },
                "last_name": {
                    "type": "text"
                }
            }
        }
    }
}

es.indices.delete(index=OBJECT_INDEX, ignore_unavailable=True)
es.indices.create(index=OBJECT_INDEX,mappings=mappings, settings=settings)

/tmp/ipykernel_13388/2306691755.py:18: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index=OBJECT_INDEX, ignore_unavailable=True)
/tmp/ipykernel_13388/2306691755.py:19: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index=OBJECT_INDEX,mappings=mappings, settings=settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'object_index'})

In [8]:
document = {
    "author": {
        "first_name": "Imad",
        "last_name": "Saddik"
    }
}

response = es.index(index=OBJECT_INDEX, body=document)
response

/tmp/ipykernel_13388/2948351030.py:8: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.index(index=OBJECT_INDEX, body=document)


ObjectApiResponse({'_index': 'object_index', '_type': '_doc', '_id': 'q6cKI5UB6zEJPjityqQv', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

### 2.2 Flattened object

In [9]:
FLATTENDED_OBJECT_INDEX = "flattened_object_index"

mappings = {
    "properties": {
        "author": {
            "type": "flattened"
        }
    }
}

es.indices.delete(index=FLATTENDED_OBJECT_INDEX, ignore_unavailable=True)
es.indices.create(index=FLATTENDED_OBJECT_INDEX, mappings=mappings, settings=settings)

/tmp/ipykernel_13388/1108609983.py:11: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index=FLATTENDED_OBJECT_INDEX, ignore_unavailable=True)
/tmp/ipykernel_13388/1108609983.py:12: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index=FLATTENDED_OBJECT_INDEX, mappings=mappings, settings=settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'flattened_object_index'})

In [10]:
document = {
    "author": {
        "first_name": "Imad",
        "last_name": "Saddik"
    }
}
response = es.index(index=FLATTENDED_OBJECT_INDEX, body=document)
response

/tmp/ipykernel_13388/3407036014.py:7: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.index(index=FLATTENDED_OBJECT_INDEX, body=document)


ObjectApiResponse({'_index': 'flattened_object_index', '_type': '_doc', '_id': 'rKcOI5UB6zEJPjitMaSx', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

### 2.3 Nested object

In [11]:
NESTED_OBJECT_INDEX = "nested_object_index"

mappings = {
    "properties": {
        "user": {
            "type": "nested",
        }
    }
}

es.indices.delete(index=NESTED_OBJECT_INDEX, ignore_unavailable=True)
es.indices.create(index=NESTED_OBJECT_INDEX,mappings=mappings, settings=settings)

/tmp/ipykernel_13388/701800038.py:11: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index=NESTED_OBJECT_INDEX, ignore_unavailable=True)
/tmp/ipykernel_13388/701800038.py:12: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index=NESTED_OBJECT_INDEX,mappings=mappings, settings=settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'nested_object_index'})

In [12]:
documents = [
    {
        "first": "John",
        "last": "Smith"
    },
    {
        "first": "Imad",
        "last": "Saddik"
    }
]
response = es.index(index=NESTED_OBJECT_INDEX, body={"user": documents})
response

/tmp/ipykernel_13388/2416284370.py:11: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.index(index=NESTED_OBJECT_INDEX, body={"user": documents})


ObjectApiResponse({'_index': 'nested_object_index', '_type': '_doc', '_id': 'racRI5UB6zEJPjitGqR4', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

## 3. Text search types

### 3.1 Text

In [13]:
TEXT_INDEX = "text_index"

mappings = {
    "properties": {
        "email_body": {
            "type": "text"
        }
    }
}

es.indices.delete(index=TEXT_INDEX, ignore_unavailable=True)
es.indices.create(index=TEXT_INDEX, mappings=mappings, settings=settings)

/tmp/ipykernel_13388/1860281493.py:11: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index=TEXT_INDEX, ignore_unavailable=True)
/tmp/ipykernel_13388/1860281493.py:12: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index=TEXT_INDEX, mappings=mappings, settings=settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'text_index'})

In [14]:
document = {
    "email_body": "Hello, this is a test email."
}
response = es.index(index=TEXT_INDEX, body=document)
response

/tmp/ipykernel_13388/326400323.py:4: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.index(index=TEXT_INDEX, body=document)


ObjectApiResponse({'_index': 'text_index', '_type': '_doc', '_id': 'rqcUI5UB6zEJPjith6Tt', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

### 3.2 Completion

In [15]:
TEXT_COMPLETION_INDEX = "text_completion_index"

mappings = {
    "properties": {
        "suggest": {
            "type": "completion"
        }
    }
}

es.indices.delete(index=TEXT_COMPLETION_INDEX, ignore_unavailable=True)
es.indices.create(index=TEXT_COMPLETION_INDEX, mappings=mappings, settings=settings)

/tmp/ipykernel_13388/2023398724.py:11: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index=TEXT_COMPLETION_INDEX, ignore_unavailable=True)
/tmp/ipykernel_13388/2023398724.py:12: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index=TEXT_COMPLETION_INDEX, mappings=mappings, settings=settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'text_completion_index'})

In [16]:
document_1 = {
    "suggest": {
        "input": ["Mars", "Planet"]
    }
}

document_2 = {
    "suggest": {
        "input": ["Andromeda", "Galaxy"]
    }
}

es.index(index=TEXT_COMPLETION_INDEX, body=document_1)
es.index(index=TEXT_COMPLETION_INDEX, body=document_2)

/tmp/ipykernel_13388/3579025236.py:13: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index=TEXT_COMPLETION_INDEX, body=document_1)
/tmp/ipykernel_13388/3579025236.py:14: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index=TEXT_COMPLETION_INDEX, body=document_2)


ObjectApiResponse({'_index': 'text_completion_index', '_type': '_doc', '_id': 'sKcXI5UB6zEJPjitQKRV', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1})

## 4. Spatial data types

### 4.1 Geo point

In [20]:
GEO_POINT_INDEX = "geo_point_index"

mappings = {
    "properties": {
        "location": {
            "type": "geo_point"
        }
    }
}

es.indices.delete(index=GEO_POINT_INDEX, ignore_unavailable=True)
es.indices.create(index=GEO_POINT_INDEX,mappings=mappings, settings=settings)

/tmp/ipykernel_13388/1713042326.py:14: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index=GEO_POINT_INDEX, ignore_unavailable=True)
/tmp/ipykernel_13388/1713042326.py:15: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index=GEO_POINT_INDEX,mappings=mappings, settings=settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'geo_point_index'})

In [24]:
document = {
    "desc": "Geopoint as an object using GeoJSON format",
    "location": [ -71.34, 41.12]
}
response = es.index(index=GEO_POINT_INDEX, body=document)
response

/tmp/ipykernel_13388/1036191643.py:5: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.index(index=GEO_POINT_INDEX, body=document)


ObjectApiResponse({'_index': 'geo_point_index', '_type': '_doc', '_id': 'tachI5UB6zEJPjit5KTk', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

### 4.2 Geo shape

In [25]:
GEO_SHAPE_INDEX = "geo_shape_index"

mappings = {
    "properties": {
        "location": {
            "type": "geo_shape"
        }
    }
}

es.indices.delete(index=GEO_SHAPE_INDEX, ignore_unavailable=True)
es.indices.create(index=GEO_SHAPE_INDEX,mappings=mappings, settings=settings)

/tmp/ipykernel_13388/1222328904.py:11: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index=GEO_SHAPE_INDEX, ignore_unavailable=True)
/tmp/ipykernel_13388/1222328904.py:12: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index=GEO_SHAPE_INDEX,mappings=mappings, settings=settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'geo_shape_index'})

In [26]:
document_1 = {
    "location": {
        "type": "LineString",
        "coordinates": [
            [
                -77.03653,
                38.897676
            ],
            [
                -77.009051,
                38.889939
            ]
        ]
    }
}
document_2 = {
    "location": {
        "type": "Polygon",
        "coordinates": [
            [
                [100, 0],
                [101, 0],
                [101, 1],
                [100, 1],
                [100, 0],
            ],
            [
                [100.2, 0.2],
                [100.8, 0.2],
                [100.8, 0.8],
                [100.2, 0.8],
                [100.2, 0.2],
            ]
        ]
    }
}

es.index(index=GEO_SHAPE_INDEX, body=document_1)
es.index(index=GEO_SHAPE_INDEX, body=document_2)

/tmp/ipykernel_13388/624711158.py:38: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index=GEO_SHAPE_INDEX, body=document_1)
/tmp/ipykernel_13388/624711158.py:39: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index=GEO_SHAPE_INDEX, body=document_2)


ObjectApiResponse({'_index': 'geo_shape_index', '_type': '_doc', '_id': 't6ckI5UB6zEJPjitfKSz', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1})

### 4.3 Point

In [27]:
POINT_INDEX = "point_index"

mappings = {
    "properties": {
        "location": {
            "type": "point"
        }
    }
}

es.indices.delete(index=POINT_INDEX, ignore_unavailable=True)
es.indices.create(index=POINT_INDEX, mappings=mappings, settings=settings)

/tmp/ipykernel_13388/2051410063.py:11: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index=POINT_INDEX, ignore_unavailable=True)
/tmp/ipykernel_13388/2051410063.py:12: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index=POINT_INDEX, mappings=mappings, settings=settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'point_index'})

In [ ]:
document = {
    "location": [-71.34, 41.12]
}

response = es.index(index=POINT_INDEX, body=document)
response

/tmp/ipykernel_13388/162290521.py:8: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.index(index=POINT_INDEX, body=document)


ObjectApiResponse({'_index': 'point_index', '_type': '_doc', '_id': 'uacnI5UB6zEJPjitcKR6', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})